## Data for this assignment is taken from a Kaggle contest: https://www.kaggle.com/c/vietai-advance-course-retinal-disease-detection/overview
Explanation of the data set:
The training data set contains 3435 retinal images that represent multiple pathological disorders. The patholgy classes and corresponding labels are: included in 'train.csv' file and each image can have more than one class category (multiple pathologies).
The labels for each image are

```
-opacity (0), 
-diabetic retinopathy (1), 
-glaucoma (2),
-macular edema (3),
-macular degeneration (4),
-retinal vascular occlusion (5)
-normal (6)
```
The test data set contains 350 unlabelled images.

For this assignment, you are working with specialists for Diabetic Retinopathy and Glaucoma only, and your client is interested in a predictive learning model along with feature explanability and self-learning for Diabetic Retinopathy and Glaucoma vs. Normal images.


Design models and methods for the following tasks. Each task should be accompanied by code, plots/images (if applicable), tables (if applicable) and text:
## Task 1: Build a classification model for Diabetic Retinopathy and Glaucoma vs normal images. You may consider multi-class classification vs. all-vs-one classification. Clearly state your choice and share details of your model, paremeters and hyper-paramaterization pprocess. (60 points)
```
a. Perform 70/30 data split and report performance scores on the test data set.
b. You can choose to apply any data augmentation strategy. 
Explain your methods and rationale behind parameter selection.
c. Show Training-validation curves to ensure overfitting and underfitting is avoided.
```


In [16]:
import pathlib 


DATA_DIR = pathlib.Path.cwd() / "data" / "train"

labels_df = pd.read_csv("data/train.csv")
labels_df["image_path"] = labels_df["filename"].apply(lambda x: str(DATA_DIR / x))

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(labels_df, random_state=42, shuffle=True, test_size=0.3)

In [26]:
classes = ["opacity", "diabetic retinopathy", "glaucoma", "macular edema", "macular degeneration", "retinal vascular occlusion", "normal"]

In [27]:
X_train, y_train = train["image_path"].values, train[classes].values
X_test, y_test = train["image_path"].values, train[classes].values

In [145]:
import tensorflow as tf
import tensorflow_hub as hub

IMG_SIZE = 128
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1024
NUM_CLASSES = len(classes)
AUTOTUNE = tf.data.experimental.AUTOTUNE

def parse_function(filename, channels=3):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=channels)
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    image_normalized = image_resized / 255.0
    return image_normalized

def create_ds(X, y, is_training=True):
    images = tf.data.Dataset.from_tensor_slices((X)).map(parse_function)
    labels = tf.data.Dataset.from_tensor_slices((y))
    dataset = tf.data.Dataset.zip((images, labels))
    
    if is_training:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(
            buffer_size=SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True
        )

    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

In [146]:
train_ds = create_ds(X_train, y_train)
test_ds = create_ds(X_test, y_test, is_training=False)

In [147]:
for i, l in train_ds.take(1):
    print(i.shape, l.shape)

(64, 128, 128, 3) (64, 7)


In [181]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers.experimental import preprocessing

LR_INIT = 0.001
MOMENTUM = 0.99

model = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal_and_vertical"),
    ResNet50(include_top=False, classes=NUM_CLASSES),
    tf.keras.layers.Dense(40),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.build([None, IMG_SIZE, IMG_SIZE, 3])  # Batch input shape.

model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, name="categorical_crossentropy"),
        optimizer=tf.keras.optimizers.SGD(learning_rate=LR_INIT, momentum=MOMENTUM),
        metrics=[
                *[
                    tf.keras.metrics.Recall(
                        class_id=int(class_id), name=f"{name}_recall"
                    )
                    for class_id, name in enumerate(classes)
                ],
                *[
                    tf.keras.metrics.Precision(
                        class_id=int(class_id), name=f"{name}_precision"
                    )
                    for class_id, name in enumerate(classes)
                ],
                "accuracy",
            ],
    )

In [182]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_flip_14 (RandomFlip)  (None, 128, 128, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
dense_21 (Dense)             (None, 4, 4, 40)          81960     
_________________________________________________________________
flatten_5 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 7)                 4487      
Total params: 23,674,159
Trainable params: 23,621,039
Non-trainable params: 53,120
_________________________________________________________________


In [184]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_Confidences_accuracy",
    verbose=1,
    patience=20,
    mode="max",
    restore_best_weights=True,
)

model.fit(train_ds, validation_data=test_ds, epochs=100, callbacks=[early_stopping], verbose=1)

Epoch 1/100
 3/38 [=>............................] - ETA: 2:33 - loss: 3.9613 - opacity_recall: 0.5765 - diabetic retinopathy_recall: 0.1538 - glaucoma_recall: 0.0769 - macular edema_recall: 0.0769 - macular degeneration_recall: 0.1212 - retinal vascular occlusion_recall: 0.0476 - normal_recall: 0.5000 - opacity_precision: 0.5904 - diabetic retinopathy_precision: 0.2222 - glaucoma_precision: 0.3333 - macular edema_precision: 0.5000 - macular degeneration_precision: 0.2857 - retinal vascular occlusion_precision: 0.3333 - normal_precision: 0.7895 - accuracy: 0.4375       

KeyboardInterrupt: 


## Task 2: Visualize the heatmap/saliency/features using any method of your choice to demonstrate what regions of interest contribute to Diabetic Retinopathy and Glaucoma, respectively. (25 points)
```
Submit images/folder of images with heatmaps/features aligned on top of the images, or corresponding bounding boxes, and report what regions of interest in your opinion represent the pathological sites.
```

In [160]:
glaucoma_sample_images = train[train["glaucoma"] == 1].image_path.sample(5).values
diabetic_retinopathy_sample_images = train[train["diabetic retinopathy"] == 1].image_path.sample(5).values

In [161]:
classes.index("glaucoma")

2

In [164]:
from tf_explain.core.grad_cam import GradCAM
# Load a sample image (or multiple ones)
img = parse_function(glaucoma_sample_images[0])
data = ([img.numpy()], None)

# Start explainer
explainer = GradCAM()
grid = explainer.explain(data, model, class_index=2)  # 281 is the tabby cat index in ImageNet

explainer.save(grid, "./images", "glaucoma.png")

/Users/nshpak/code/repos/retina/venv/lib/python3.7/site-packages/tf_explain/utils/display.py:112: RuntimeWarning: invalid value encountered in true_divide
  heatmap = (heatmap - np.min(heatmap)) / (heatmap.max() - heatmap.min())



## Task 3: Using the unlabelled data set in the 'test' folder augment the training data (semi-supervised learning) and report the variation in classification performance on test data set.(15 points)
[You may use any method of your choice, one possible way is mentioned below.] 

```
Hint: 
a. Train a model using the 'train' split.
b. Pass the unlabelled images through the trained model and retrieve the dense layer feature prior to classification layer. Using this dense layer as representative of the image, apply label propagation to retrieve labels correspndng to the unbalelled data.
c. Next, concatenate the train data with the unlabelled data (that has now been self labelled) and retrain the network.
d. Report classification performance on test data
Use the unlabelled test data  to improve classification performance by using a semi-supervised label-propagation/self-labelling approach. (20 points)
```

## [Hint: If you are wondering how to use the "dense layer representative of an image" in step 2, see this exercise that extracts a [1,2048] dense representattive from an image using the InceptionV3 pre-trained model.]
https://colab.research.google.com/drive/14-6qRGARgBSj4isZk86zKQtyIT2f9Wu1#scrollTo=_IqraxtP4Ex3


## Good Luck!